<a href="https://colab.research.google.com/github/kkushal05/tutorials/blob/main/spark_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%bash

sudo apt install -y mongodb >log
service mongodb start

 * Starting database mongodb
   ...done.




debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 8.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [3]:
%%bash

ps -ef | grep mongo

mongodb      485       1 68 14:04 ?        00:00:00 /usr/bin/mongod --config /etc/mongodb.conf


In [4]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() 

['admin', 'local']

In [ ]:
!pip install pandas
!pip install pyspark
!pip install pymongo
!pip install sodapy
!pip install progressbar

In [ ]:
import pandas as pd
from sodapy import Socrata
import pymongo
from pymongo import MongoClient
import pyspark
from pyspark.sql import SparkSession
import progressbar
from pyspark.sql import Row
import os

client = Socrata("data.cityofnewyork.us", None)
start = 0
chunk_size = 50
results = []
spark = SparkSession.builder.appName('sparkdf').getOrCreate()

spark = SparkSession \
    .builder \
    .appName("mongodbtest1") \
    .master('local')\
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1:27017/nypd_raw_data") \
    .config("spark.mongodb.output.uri", "mongodb:/127.0.0.1:27017/nypd_raw_data") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .getOrCreate()

mongo_client = MongoClient()
db = mongo_client['raw_data']

def load_data_for_nypd_call_for_service():
  start = 0
  chunk_size = 50
  data_id = "n2zq-pubd";
  cols = "NYPD_PCT_CD, BORO_NM, TYP_DESC, ARRIVD_TS, ADD_TS"
  db.call_for_service.drop()
  record_count = client.get(data_id, select="COUNT(*)")

  while True: 
    results = client.get(data_id, select=cols, offset=start, limit=chunk_size)
    dataframe = spark.createDataFrame(results)
    dataframe = dataframe.na.drop()
    for item in dataframe.collect():
      db.call_for_service.insert_one(item.asDict())
    print("Record count in  DB : " + str(db.call_for_service.count_documents({})))
    start = start + chunk_size
    if start > int(record_count[0]['COUNT']):
        break

def load_data_for_ems():
  start = 0
  chunk_size = 50
  data_id = "76xm-jjuj";
  cols = "INCIDENT_DATETIME, BOROUGH, POLICEPRECINCT, FIRST_ON_SCENE_DATETIME"
  record_count = client.get(data_id, select="COUNT(*)")
  db.ems.drop()

  while True: 
    results = client.get(data_id, select=cols, offset=start, limit=chunk_size)
    dataframe = spark.createDataFrame(results)
    dataframe = dataframe.na.drop()
    for item in dataframe.collect():
      db.ems.insert_one(item.asDict())
    print("Record count in  DB : " + str(db.ems.count_documents({})))
    start = start + chunk_size
    if start > int(record_count[0]['COUNT']):
        break

def load_data_for_fire_dept():
  start = 0
  chunk_size = 50
  data_id = "8m42-w767";
  cols = "INCIDENT_DATETIME, INCIDENT_BOROUGH, POLICEPRECINCT, FIRST_ON_SCENE_DATETIME, INCIDENT_CLASSIFICATION, INCIDENT_RESPONSE_SECONDS_QY"
  record_count = client.get(data_id, select="COUNT(*)")
  db.fire_dept.drop()

  while True: 
    results = client.get(data_id, select=cols, offset=start, limit=chunk_size)
    dataframe = spark.createDataFrame(results)
    dataframe = dataframe.na.drop()
    for item in dataframe.collect():
      db.fire_dept.insert_one(item.asDict())
    print("Record count in  DB : " + str(db.fire_dept.count_documents({})))
    start = start + chunk_size
    if start > int(record_count[0]['COUNT']):
        break

load_data_for_nypd_call_for_service()
load_data_for_fire_dept()
load_data_for_ems()

#dataframe.show()